In [ ]:
#|default_exp callbacks
#|default_cls_lvl 3

In [ ]:
#| nbflags skip_exec


In [ ]:
#|hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# callbacks

> Callbacks used by the BLURR library.

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
#|export
import importlib, sys, torch
from typing import Any, Callable, Dict, List, Optional, Union, Type

from fastcore.all import *
from fastai.callback.all import *
from fastai.imports import *
from fastai.learner import *
from fastai.torch_core import *
from transformers import PreTrainedModel

In [ ]:
#| echo: false
import gc, pdb

import GPUtil as GPU
from IPython.display import display
from fastai.text.all import *
from fastcore.test import *
from nbdev.showdoc import show_doc

from blurr.text.modeling.all import *


In [ ]:
#|hide
#|cuda

nvidia_smi_idx = 2
torch.cuda.set_device(1)
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")


Using GPU #1: GeForce GTX 1080 Ti


## Gradient Checkpointing

In [ ]:
#|export
class CheckpointingNotSupported(Exception):
    def __init__(self, msg="Model does not support gradient checkpointing."):
        super().__init__(msg)

In [ ]:
#|export
class GradientCheckpointing(Callback):
    """A fastai callback to enable gradient checkpointing for compatible HuggingFace models."""

    def before_fit(self):
        """Enable gradient checkpointing on before_fit event."""
        
        # Check that huggingface model supports gradient checkpointing
        if not self.model.hf_model.supports_gradient_checkpointing:
            raise CheckpointingNotSupported()
            
        if self.model.hf_model.is_gradient_checkpointing == False:
            self.model.hf_model.gradient_checkpointing_enable()
            
    def after_fit(self):
        """Disable gradient checkpointing on after_fit event."""
        if self.model.hf_model.is_gradient_checkpointing:
            self.model.hf_model.gradient_checkpointing_disable()
    
    @staticmethod
    def supported(model: PreTrainedModel):
        """Tests whether a HuggingFace `PreTrainedModel` supports gradient checkpointing."""
        return model.supports_gradient_checkpointing

In [ ]:
def clear_memory():
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
def gpu_memory(device_idx=nvidia_smi_idx):
    return GPU.getGPUs()[device_idx].memoryUsed

In [ ]:
# step 1: load Data
path = untar_data(URLs.IMDB_SAMPLE)
model_path = Path("models")
imdb_df = pd.read_csv(path / "texts.csv")

In [ ]:
# step 2: create Learner
learn = BlearnerForSequenceClassification.from_data(
    imdb_df, 
    "roberta-large", 
    dl_kwargs={"bs": 4}
)

In [ ]:
# train for a single epoch for baseline memory usage
learn.fit_one_cycle(1, lr_max=1e-3)

base_mem = gpu_memory()
print(f"{base_mem} MBs used.")

# Clear gpu memory
clear_memory()

epoch,train_loss,valid_loss,f1_score,accuracy,time
0,0.350596,0.233957,0.918033,0.925000,00:57


9585.0 MBs used.


In [ ]:
# train with GradientCheckpointing
learn.fit_one_cycle(1, lr_max=1e-3, cbs=[GradientCheckpointing()])

check_mem = gpu_memory()
print(f"{check_mem} MBs used.")

test_eq(base_mem > check_mem, True)

epoch,train_loss,valid_loss,f1_score,accuracy,time
0,0.207228,0.226048,0.918033,0.925000,01:21


3017.0 MBs used.


## Export -

In [ ]:
#|hide
from nbdev import nbdev_export

nbdev_export()


Converted 00_callbacks.ipynb.
Converted 00_utils.ipynb.
Converted 01_text-callbacks.ipynb.
Converted 01_text-utils.ipynb.
Converted 11_text-data-core.ipynb.
Converted 11_text-modeling-core.ipynb.
Converted 12_text-data-language-modeling.ipynb.
Converted 12_text-modeling-language-modeling.ipynb.
Converted 13_text-data-token-classification.ipynb.
Converted 13_text-modeling-token-classification.ipynb.
Converted 14_text-data-question-answering.ipynb.
Converted 14_text-modeling-question-answering.ipynb.
Converted 20_text-data-seq2seq-core.ipynb.
Converted 20_text-modeling-seq2seq-core.ipynb.
Converted 21_text-data-seq2seq-summarization.ipynb.
Converted 21_text-modeling-seq2seq-summarization.ipynb.
Converted 22_text-data-seq2seq-translation.ipynb.
Converted 22_text-modeling-seq2seq-translation.ipynb.
Converted 99a_text-examples-high-level-api.ipynb.
Converted 99b_text-examples-glue.ipynb.
Converted 99c_text-examples-glue-plain-pytorch.ipynb.
Converted 99d_text-examples-multilabel.ipynb.
Conv